In [1]:
# titanic
# PCA component 수 변화 
# 분류 정확도, 설명력

In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot

In [3]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
titanic.drop(columns=['deck'],inplace=True) # 결측치가 너무 많아 열 삭제
titanic.embarked.fillna('S', inplace=True) # 결측치가 별로 없어서 최빈값으로 대체
titanic.embark_town.fillna('Southampton',inplace=True) # 상동
titanic.age.fillna(titanic.age.median().round(1),inplace=True) # 중앙값으로 대체
titanic.isna().sum().sum()

0

In [6]:
# 중복제거 
# class       pclass
# embark_town     embarked
# alive       survived
# alone       sibsp
# adult_male

In [7]:
df = titanic[['survived','pclass','sex','age','sibsp','parch','fare','embarked']]
df.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
886,0,2,male,27.0,0,0,13.00,S
887,1,1,female,19.0,0,0,30.00,S
888,0,3,female,28.0,1,2,23.45,S
889,1,1,male,26.0,0,0,30.00,C
890,0,3,male,32.0,0,0,7.75,Q


In [8]:
# 카테고리형 데이터를 숫자로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [9]:
df.sex = le.fit_transform(df.sex)
df.tail()

C:\Users\YONSAI\AppData\Local\Temp\ipykernel_5480\4199047631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sex = le.fit_transform(df.sex)


,survived,pclass,sex,age,sibsp,parch,fare,embarked
886,0,2,1,27.0,0,0,13.00,S
887,1,1,0,19.0,0,0,30.00,S
888,0,3,0,28.0,1,2,23.45,S
889,1,1,1,26.0,0,0,30.00,C
890,0,3,1,32.0,0,0,7.75,Q


In [10]:
# 여자0, 남자1
df.sex.value_counts()

sex
1    577
0    314
Name: count, dtype: int64

In [11]:
df.embarked.value_counts()

embarked
S    646
C    168
Q     77
Name: count, dtype: int64

In [12]:
df.embarked = le.fit_transform(df.embarked)

C:\Users\YONSAI\AppData\Local\Temp\ipykernel_5480\1801681732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.embarked = le.fit_transform(df.embarked)


In [13]:
# C:0, Q:1, S:2
df.embarked.value_counts()

embarked
2    646
0    168
1     77
Name: count, dtype: int64

In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [15]:
# data, 정답
X = df.iloc[:, 1:].values
y = df.survived.values

In [16]:
from sklearn.preprocessing import StandardScaler 
X_std = StandardScaler().fit_transform(X)

In [17]:
# Scikit-Learn 으로 공분산 구하지않고 numpy로 구함.. 선생님은 사이킷런으로했는데 더 정확함
X_cov = np.cov(X_std.T)
X_cov

array([[ 1.0011236 ,  0.13204869, -0.34028024,  0.08317471,  0.01846339,
        -0.55011704,  0.16227994],
       [ 0.13204869,  1.0011236 ,  0.08125373, -0.11475961, -0.24576479,
        -0.1825377 ,  0.10838385],
       [-0.34028024,  0.08125373,  1.0011236 , -0.23355846, -0.17267575,
         0.09679706, -0.01877539],
       [ 0.08317471, -0.11475961, -0.23355846,  1.0011236 ,  0.41530381,
         0.15983043,  0.06830695],
       [ 0.01846339, -0.24576479, -0.17267575,  0.41530381,  1.0011236 ,
         0.21646789,  0.03984311],
       [-0.55011704, -0.1825377 ,  0.09679706,  0.15983043,  0.21646789,
         1.0011236 , -0.22497186],
       [ 0.16227994,  0.10838385, -0.01877539,  0.06830695,  0.03984311,
        -0.22497186,  1.0011236 ]])

In [19]:
eig_vals, eig_vecs = np.linalg.eig(X_cov)
eig_vals

array([1.85796677, 1.71653366, 0.36689739, 0.99749852, 0.66716428,
       0.55771075, 0.84409379])

In [20]:
eig_vecs

array([[ 0.53352353,  0.34389883,  0.68835705,  0.2003863 ,  0.28673039,
        -0.02942797,  0.00397617],
       [ 0.34356535, -0.22158262, -0.07602253, -0.31123395,  0.01243361,
        -0.24758768, -0.81777977],
       [-0.14488859, -0.49989395,  0.20010723, -0.40852423,  0.68599662,
         0.1386299 ,  0.17991326],
       [-0.18812291,  0.5412223 , -0.10314171, -0.24011808,  0.19048691,
         0.67828452, -0.32716684],
       [-0.29120493,  0.51409535, -0.15871747, -0.19220909,  0.37130869,
        -0.67058141,  0.03493604],
       [-0.61439189, -0.04708266,  0.65950535, -0.09066006, -0.3263448 ,
        -0.09307198, -0.24894968],
       [ 0.28045472,  0.15267088,  0.09780328, -0.77023791, -0.40790121,
        -0.01429632,  0.35863219]])

In [22]:
PC1 = np.dot(X_std, eig_vecs[:, :1])
PC1[:5]

array([[ 1.30639006],
       [-2.3691106 ],
       [ 0.70501784],
       [-1.40170678],
       [ 1.32243411]])

In [24]:
PC2 = np.dot(X_std, eig_vecs[:, 1:2]) # 1:2 data 안겹치려고? 
PC2[:5]

array([[ 0.50772049],
       [-0.91292681],
       [ 0.3261736 ],
       [-0.39444756],
       [-0.48352779]])

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

def pca_accuracy(X, y):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=2023
    )
    rfc = RandomForestClassifier(random_state=2023)
    rfc.fit(X_train, y_train)
    score = rfc.score(X_test, y_test)
    return score

In [27]:
from sklearn.decomposition import PCA
for n in [2, 4, 6]:
    pca = PCA(n_components=n)
    X_pca = pca.fit_transform(X_std)
    explained = sum(pca.explained_variance_ratio_)
    acc = pca_accuracy(X_pca, df.survived.values)
    print(f'PCA{n}: 설명력 - {explained:.4f}, 정확도 - {acc:.4f}')

PCA2: 설명력 - 0.5101, 정확도 - 0.7318
PCA4: 설명력 - 0.7729, 정확도 - 0.7374
PCA6: 설명력 - 0.9476, 정확도 - 0.7598
